In [231]:
import pandas as pd
import numpy as np
import os
os.getcwd()

'C:\\Users\\Meenu\\Documents\\SellAnyCar'

In [232]:
os.chdir('C:\\Users\\Meenu\\Documents\\SellAnyCar')

In [233]:
df = pd.read_excel('Data.xlsx',sheet_name='RawData')

In [234]:
df.isnull().sum()

Customer Number                0
Customer Country               0
Year                           0
Month                          0
Shipment Destination Region    0
Revenue (EUR)                  0
dtype: int64

In [235]:

df['Month'] = df['Month'].astype(str)
df['month'] = df['Month'].str[-2:].astype(int)
df.loc[(df['month']>=1) & (df['month']<=3),'quarter'] = 'Q1'
df.loc[(df['month']>=4) & (df['month']<=6),'quarter'] = 'Q2'
df.loc[(df['month']>=7) & (df['month']<=9),'quarter'] = 'Q3'
df.loc[(df['month']>=10) & (df['month']<=12),'quarter'] = 'Q4'

In [236]:
quarterly_segmented = df.groupby(['Customer Number','Customer Country','Year','quarter','month'])['Revenue (EUR)'].sum().reset_index()

In [237]:
jf = quarterly_segmented
jf = jf.groupby('Customer Number')['Revenue (EUR)'].pct_change()*100
jf = jf.reset_index().drop(columns=['index']).rename(columns={'Revenue (EUR)':'pct_chng'})

In [238]:
quarterly_segmented = quarterly_segmented.join(jf)
quarterly_segmented.fillna(0,inplace=True)

In [239]:
buy_2018 = quarterly_segmented.loc[(quarterly_segmented['Year']== 2018)] 
buy_q3_2018 = buy_2018[buy_2018['quarter']=='Q3'] 
buy_q3_2018['last Q'] = 'Y'

<ipython-input-239-b328e8086430>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_q3_2018['last Q'] = 'Y'


In [240]:
ff = pd.merge(df[['Customer Number']],buy_q3_2018,on=['Customer Number'],how='left').drop_duplicates()
ff[ff['last Q']=='Y'].nunique()
ff['last Q'] = ff['last Q'].fillna('N')

In [241]:
ff = ff.drop_duplicates(subset='Customer Number')

In [242]:
df.loc[df['Customer Number'].isin(buy_q3_2018['Customer Number']),'Bought on last Q'] = "Y" 
df.loc[(df['Bought on last Q']!='Y'),'Bought on last Q'] = 'N'

In [243]:
bought_on_q3_2018 = pd.DataFrame(buy_q3_2018['Customer Number'].unique())

In [244]:
gg =  df.groupby(['Customer Number','Customer Country','Year','quarter'])['Revenue (EUR)'].sum().reset_index()
ff = gg[~((gg['Year'] == 2018) & (gg['quarter']=='Q3'))]

In [245]:
grouped = df[df['Bought on last Q']=='Y'].groupby(['Customer Number','Customer Country','Year','quarter'])['Revenue (EUR)'].sum().reset_index()

In [246]:
num_orders = grouped['Customer Number'].value_counts().reset_index().rename(columns={'index':'Customer Number','Customer Number':'quarter count'})

In [247]:
grouped = pd.merge(grouped,num_orders,on='Customer Number')

In [248]:
grouped.loc[grouped['quarter count']==1,'Only Last Q'] = 'Y'


In [249]:
only_last_q = grouped.loc[(grouped['quarter count']==1) & (grouped['Year']==2018) & (grouped['quarter']=='Q3')]

In [250]:
df.loc[(df['Customer Number'].isin(only_last_q['Customer Number'])),'Only Last Q'] = 'Y'

In [251]:
df['Only Last Q'] = df['Only Last Q'].fillna('N')

In [252]:
q3_q2_2018 = df.loc[(df['Year']==2018)  & ((df['quarter']=='Q3')|(df['quarter']=='Q2'))]
q3_2018 = df.loc[(df['Year']==2018)  & ((df['quarter']=='Q3'))]

not_q3_2018 = df.loc[~((df['Year']==2018)  & ((df['quarter']=='Q3')))]
not_q3_2018 = not_q3_2018.groupby('Customer Number')['Revenue (EUR)'].sum().reset_index().rename(columns={'Revenue (EUR)':'Last 6 Q Revenue'})
q3_2018 = q3_2018.groupby('Customer Number')['Revenue (EUR)'].sum().reset_index().rename(columns={'Revenue (EUR)':'Last Q Revenue'})

In [253]:
rest_of_grouped =  df.loc[~((df['Year']==2018)  & ((df['quarter']=='Q3')|(df['quarter']=='Q2')))]

In [254]:
last_2q_revenue = q3_q2_2018.groupby('Customer Number')['Revenue (EUR)'].sum().reset_index().rename(columns={'Revenue (EUR)':'Last 2 Q Revenue'})

In [255]:
first_q_revenues = rest_of_grouped.groupby('Customer Number')['Revenue (EUR)'].sum().reset_index().rename(columns={'Revenue (EUR)':'First 5 Q Revenue'})

In [256]:
agg_quarterly_revenue = pd.merge(first_q_revenues,last_2q_revenue,how='outer').fillna(0)
agg_quarterly_revenue = pd.merge(agg_quarterly_revenue,not_q3_2018,how='outer').fillna(0)
agg_quarterly_revenue = pd.merge(agg_quarterly_revenue,q3_2018,how='outer').fillna(0)

In [257]:
agg_quarterly_revenue['avg_qtrly_rev_5'] = agg_quarterly_revenue['First 5 Q Revenue']/5
agg_quarterly_revenue['avg_qtrly_rev_2'] = agg_quarterly_revenue['Last 2 Q Revenue']/2

In [258]:
agg_quarterly_revenue['pct_chng'] = round((agg_quarterly_revenue['avg_qtrly_rev_2'] - agg_quarterly_revenue['avg_qtrly_rev_5'])/agg_quarterly_revenue['avg_qtrly_rev_2'],4)*100

In [259]:
agg_quarterly_revenue['pct_chng'] = agg_quarterly_revenue['pct_chng'].replace(-np.inf, np.nan)

In [260]:
agg_quarterly_revenue['pct_chng'].mean()

-233.2913903743317

In [261]:
agg_quarterly_revenue['Customer Number'].nunique()

647

In [262]:
agg_quarterly_revenue.loc[(agg_quarterly_revenue['pct_chng'] < \
                           agg_quarterly_revenue['pct_chng'].median()),'Segment'] = 'Down-Trader'

In [263]:
agg =  agg_quarterly_revenue.copy()

agg.loc[(agg['Last 6 Q Revenue']!=0.00) & (agg['Last Q Revenue']==0.00),'Segment'] = 'Lost Customer'
agg.loc[(agg['pct_chng'] < agg['pct_chng'].median()),'Segment'] = 'Down-Trader'
agg.loc[(agg['pct_chng'] > 0.00),'Segment'] = 'Growth Customers'
agg.loc[(agg['Segment'].isnull()) & (agg['pct_chng'] >= -20),'Segment'] = "'On the verge' Customers"
agg.loc[(agg['Segment'].isnull()) & (agg['pct_chng'] <= -20),'Segment'] = "Down-Trader"
agg.loc[(agg['First 5 Q Revenue']==0.00) & (agg['Last 2 Q Revenue']!=0.00),'Segment'] = 'Newly Acquired'

In [264]:
#agg.head()

In [265]:
df1 = df[['Customer Number','Customer Country']]
print(df1)



      Customer Number Customer Country
0          ABC0000464               PT
1          ABC0000464               PT
2          ABC0000464               PT
3          ABC0000464               PT
4          ABC0000464               PT
...               ...              ...
23538      ABM0018947               DE
23539      ABM0018947               DE
23540      ABM0018947               DE
23541      ABM0018947               DE
23542      ABM0018947               DE

[23543 rows x 2 columns]


In [266]:
agg1 = pd.merge(df1,agg,on=['Customer Number'],how='left').drop_duplicates()

In [267]:
agg1.head()

,Customer Number,Customer Country,First 5 Q Revenue,Last 2 Q Revenue,Last 6 Q Revenue,Last Q Revenue,avg_qtrly_rev_5,avg_qtrly_rev_2,pct_chng,Segment
0,ABC0000464,PT,5477.80,812.72,6131.32,159.20,1095.560,406.360,-169.60,Down-Trader
62,ABC0009734,PT,5634.44,1058.10,6406.01,286.53,1126.888,529.050,-113.00,Down-Trader
85,ABC0012173,PT,4522.23,1123.22,5345.61,299.84,904.446,561.610,-61.05,Down-Trader
101,ABC0026905,PT,6993.66,2196.53,8095.81,1094.38,1398.732,1098.265,-27.36,Down-Trader
136,ABC0027135,PT,5161.58,1499.32,6196.27,464.63,1032.316,749.660,-37.70,Down-Trader
...,...,...,...,...,...,...,...,...,...,...
3258,ABC1674305,FR,5142.85,804.38,5947.23,0.00,1028.570,402.190,-155.74,Down-Trader
3274,ABC1684110,FR,3018.05,0.00,3018.05,0.00,603.610,0.000,NaN,Lost Customer
3286,ABC1709153,FR,14752.66,667.26,15419.92,0.00,2950.532,333.630,-784.37,Down-Trader
3302,ABC1709320,FR,8841.74,1593.86,10435.60,0.00,1768.348,796.930,-121.90,Down-Trader


In [268]:
downDF = agg1.loc[agg1['Segment']=="Down-Trader"]
downDF["Customer Number"].count()

413

In [269]:
lostDF = agg1.loc[agg1['Segment']=="Lost Customer"]
lostDF["Customer Number"].count()

93

In [270]:
vergeDF = agg1.loc[agg1['Segment']=="'On the verge' Customers"]
vergeDF["Customer Number"].count()

41

In [271]:
growthDF = agg1.loc[agg1['Segment']=='Growth Customers']
growthDF["Customer Number"].count()

71

In [273]:
newDF = agg1.loc[agg1['Segment']=='Newly Acquired']
newDF["Customer Number"].count()

29